# DeepLabCut Toolbox - Colab Demo on Topview Mouse Data
https://github.com/DeepLabCut/DeepLabCut

![alt text](https://images.squarespace-cdn.com/content/v1/57f6d51c9f74566f55ecf271/1559935526258-KFYZC8BDHK01ZIDPNVIX/mouse_skel_trail.gif?format=450w)

Demo supporting: Nath\*, Mathis\* et al. *Using DeepLabCut for markerless3D  pose estimation during behavior across species. Nature Protocols, 2019

This notebook demonstrates the necessary steps to use DeepLabCut on our demo data. We provide a sub-set of the mouse data from Mathis et al, 2018 Nature Neuroscience.

This demo notebook mostly shows the most simple code to train and evaluate your model, but many of the functions have additional features, so please check out the overview & the protocol paper!

This notebook illustrates how to use the cloud to:

- load demo data
- create a training set
- train a network
- evaluate a network
- analyze a novel video

## First, go to "Runtime" ->"change runtime type"->select "Python3", and then select "GPU"

As the COLAB environments were updated to CUDA 12.X and Python 3.11, we need to install DeepLabCut and TensorFlow in a distinct way to get TensorFlow to connect to the GPU.

In [28]:
# Install TensorFlow, tensorpack and tf_slim versions compatible with DeepLabCut
%pip install "tensorflow==2.10" "tensorpack>=0.11" "tf_slim>=1.1.0"

  Using cached gast-0.4.0-py3-none-any.whl.metadata (1.1 kB)
  Using cached google_auth-2.38.0-py2.py3-none-any.whl.metadata (4.8 kB)
  Using cached cachetools-5.5.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached pyasn1_modules-0.4.1-py3-none-any.whl.metadata (3.5 kB)
  Using cached rsa-4.9-py3-none-any.whl.metadata (4.2 kB)
  Using cached requests_oauthlib-2.0.0-py2.py3-none-any.whl.metadata (11 kB)
  Using cached pyasn1-0.6.1-py3-none-any.whl.metadata (8.4 kB)
  Using cached oauthlib-3.2.2-py3-none-any.whl.metadata (7.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.0/578.0 MB 16.3 MB/s eta 0:00:0000:0100:01
Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 14.0 MB/s eta 0:00:0000:0100:01
Using cached google_auth-2.38.0-py2.py3-none-any.whl (210 kB)
   ━━━━━━━━━━━━━━━━━━

In [4]:
# Downgrade PyTorch to a version using CUDA 11.8 and cudnn 8
# This will also install the required CUDA libraries, for both PyTorch and TensorFlow
%pip install torch torchvision 

  Using cached torch-2.6.0-cp310-cp310-manylinux1_x86_64.whl.metadata (28 kB)
  Using cached torchvision-0.21.0-cp310-cp310-manylinux1_x86_64.whl.metadata (6.1 kB)
  Using cached filelock-3.17.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.5-py3-none-any.whl.metadata (2.6 kB)
  Using cached fsspec-2025.2.0-py3-none-any.whl.metadata (11 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5

In [5]:
%pip list

Package                      Version
---------------------------- -----------
absl-py                      2.1.0
asttokens                    3.0.0
astunparse                   1.6.3
certifi                      2025.1.31
charset-normalizer           3.4.1
comm                         0.2.2
debugpy                      1.8.12
decorator                    5.1.1
exceptiongroup               1.2.2
executing                    2.1.0
filelock                     3.17.0
flatbuffers                  25.2.10
fsspec                       2025.2.0
gast                         0.6.0
google-pasta                 0.2.0
grpcio                       1.70.0
h5py                         3.13.0
idna                         3.10
importlib_metadata           8.6.1
ipykernel                    6.29.5
ipython                      8.32.0
jedi                         0.19.2
Jinja2                       3.1.5
jupyter_client               8.6.3
jupyter_core                 5.7.2
keras                        3.8

In [7]:
# As described in https://www.tensorflow.org/install/pip#step-by-step_instructions,
# create symbolic links to NVIDIA shared libraries:
!sudo ln -svf /usr/local/lib/python3.10/dist-packages/nvidia/*/lib/*.so* /usr/local/lib/python3.10/dist-packages/tensorflow

[sudo] password for asus: 
sudo: a password is required
^C


In [9]:
# Clone the entire deeplabcut repo so we can use the demo data:
!git clone -l -s https://github.com/DeepLabCut/DeepLabCut.git cloned-DLC-repo
%cd cloned-DLC-repo
!ls

fatal: destination path 'cloned-DLC-repo' already exists and is not an empty directory.
/home/asus/Downloads/cloned-DLC-repo
AUTHORS		    CONTRIBUTING.md	 examples	   setup.py
build		    deeplabcut		 LICENSE	   tests
cloned-DLC-repo     deeplabcut.egg-info  NOTICE.yml	   testscript_cli.py
CODE_OF_CONDUCT.md  dlc.py		 README.md	   _toc.yml
conda-environments  docker		 reinstall.sh	   tools
_config.yml	    docs		 requirements.txt


In [23]:
# %cd examples/openfield-Pranav-2018-10-30
%cd ..
!ls

/home/asus/Downloads/cloned-DLC-repo
AUTHORS		    CONTRIBUTING.md	 examples	   setup.py
build		    deeplabcut		 LICENSE	   tests
cloned-DLC-repo     deeplabcut.egg-info  NOTICE.yml	   testscript_cli.py
CODE_OF_CONDUCT.md  dlc.py		 README.md	   _toc.yml
conda-environments  docker		 reinstall.sh	   tools
_config.yml	    docs		 requirements.txt


In [24]:
# Install the latest DeepLabCut version (this will take a few minutes to install all the dependencies!)
%cd cloned-DLC-repo/
!pip install "."

/home/asus/Downloads/cloned-DLC-repo/cloned-DLC-repo
Processing /home/asus/Downloads/cloned-DLC-repo/cloned-DLC-repo
  Preparing metadata (setup.py) ... done
  Using cached dlclibrary-0.0.7-py3-none-any.whl.metadata (4.2 kB)
  Using cached filterpy-1.4.5-py3-none-any.whl
  Using cached ruamel.yaml-0.18.10-py3-none-any.whl.metadata (23 kB)
  Using cached imgaug-0.4.0-py2.py3-none-any.whl.metadata (1.8 kB)
  Using cached imageio_ffmpeg-0.6.0-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached numba-0.61.0-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (2.8 kB)
  Using cached matplotlib-3.8.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.8 kB)
  Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached pandas-2.2.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
  Using cached scikit_image-0.25.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014

In [25]:
%pip install tf_slim

Note: you may need to restart the kernel to use updated packages.


## PLEASE, click "restart runtime" from the output above before proceeding!

In [1]:
import deeplabcut

2025-02-22 21:19:20.369349: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-22 21:19:20.424312: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-22 21:19:20.426892: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-02-22 21:19:20.426900: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not ha

Loading DLC 2.3.11...


/home/asus/anaconda3/envs/DeepLabCut/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%ls cloned-DLC-repo/examples/openfield-Pranav-2018-10-30/
%ls .

config.yaml  evaluation-results/  training-datasets/
dlc-models/  labeled-data/        videos/
'=2.3.1'
 Anaconda3-2024.06-1-Linux-x86_64.sh
 BaiduNet/
 baidunetdisk_4.17.7_amd64.deb
 cloned-DLC-repo/
 code.deb
 Colab_DEMO_mouse_openfield.ipynb
 cursor-0.42.5x86_64.AppImage*
 files.tar
 import+numpy+as+np.py
'intan channel_id.xlsx'
 intan+channel_id.xlsx
 LabGym-master.zip
 LabGym_practical_guide.pdf
'load_intan_rhd_format(1).zip'
 load_intan_rhd_format.zip
 Mouse2.v1i.coco-segmentation.zip
 Mouse2.v1i.coco.zip
 Mouse2.v2i.coco-segmentation.zip
 MousePlatform.stl
 Mousetest.v1i.coco.zip
 NVIDIA-Linux-x86_64-550.144.03.run
 pycharm-community-2023.2/
 pycharm-community-2023.2.tar.gz
 pycharm-professional-2024.3.tar.gz
 rate.txt
'stizarr(1).txt'
 todesk-v4.7.2.0-amd64.deb
 v2ray-core.tar
 v2rayLui-v2.1.3.zip
 v2rayN-linux-64.deb
 WeChatLinux_x86_64.deb
 wxPython-4.2.2-cp311-cp311-linux_x86_64.whl


In [3]:
#create a path variable that links to the config file:
path_config_file = '/home/asus/Desktop/MouseTest-Wynne-2025-02-22/config.yaml'

# Loading example data set:
deeplabcut.load_demo_data(path_config_file)

This is not an official demo dataset.
Loaded, now creating training data...
The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!


## Start training:
This function trains the network for a specific shuffle of the training dataset.

In [5]:
#let's also change the display and save_iters just in case Colab takes away the GPU...
#if that happens, you can reload from a saved point. Typically, you want to train to 200,000 + iterations.
#more info and there are more things you can set: https://github.com/DeepLabCut/DeepLabCut/wiki/DOCSTRINGS#train_network

deeplabcut.train_network(path_config_file, shuffle=1, displayiters=100,saveiters=100, maxiters=10000)

#this will run until you stop it (CTRL+C), or hit "STOP" icon, or when it hits the end (default, 1.03M iterations).
#Whichever you chose, you will see what looks like an error message, but it's not an error - don't worry....

Config:
{'all_joints': [[0], [1], [2], [3]],
 'all_joints_names': ['bodypart1', 'bodypart2', 'bodypart3', 'objectA'],
 'alpha_r': 0.02,
 'apply_prob': 0.5,
 'batch_size': 1,
 'contrast': {'clahe': True,
              'claheratio': 0.1,
              'histeq': True,
              'histeqratio': 0.1},
 'convolution': {'edge': False,
                 'emboss': {'alpha': [0.0, 1.0], 'strength': [0.5, 1.5]},
                 'embossratio': 0.1,
                 'sharpen': False,
                 'sharpenratio': 0.3},
 'crop_pad': 0,
 'cropratio': 0.4,
 'dataset': 'training-datasets/iteration-0/UnaugmentedDataSet_MouseTestFeb22/MouseTest_Wynne95shuffle1.mat',
 'dataset_type': 'default',
 'decay_steps': 30000,
 'deterministic': False,
 'display_iters': 1000,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': '/home/asus/anaconda3/envs/DeepLabCut/lib/python3.10/site-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt',
 'intermediate_supervision': False

Selecting single-animal trainer
Batch Size is 1


/home/asus/anaconda3/envs/DeepLabCut/lib/python3.10/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


Loading ImageNet-pretrained resnet_50


2025-02-22 21:20:43.552153: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-02-22 21:20:43.552233: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Max_iters overwritten as 10000
Display_iters overwritten as 100
Save_iters overwritten as 100
Training parameter:
{'stride': 8.0, 'weigh_part_predictions': False, 'weigh_negatives': False, 'fg_fraction': 0.25, 'mean_pixel': [123.68, 116.779, 103.939], 'shuffle': True, 'snapshot_prefix': '/home/asus/Desktop/MouseTest-Wynne-2025-02-22/dlc-models/iteration-0/MouseTestFeb22-trainset95shuffle1/train/snapshot', 'log_dir': 'log', 'global_scale': 0.8, 'location_refinement': True, 'locref_stdev': 7.2801, 'locref_loss_weight': 0.05, 'locref_huber_loss': True, 'optimizer': 'sgd', 'intermediate_supervision': False, 'intermediate_supervision_layer': 12, 'regularize': False, 'weight_decay': 0.0001, 'crop_pad': 0, 'scoremap_dir': 'test', 'batch_size': 1, 'dataset_type': 'default', 'deterministic': False, 'mirror': False, 'pairwise_huber_loss': False, 'weigh_only_present_joints': False, 'partaffinityfield_predict': False, 'pairwise_predict': False, 'all_joints': [[0], [1], [2], [3]], 'all_joints_names

iteration: 100 loss: 0.0473 lr: 0.005
iteration: 200 loss: 0.0188 lr: 0.005
2025-02-22 21:24:24.382979: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at mkl_conv_grad_filter_ops.cc:639 : ABORTED: Operation received an exception:Status: 5, message: could not create a primitive, in file tensorflow/core/kernels/mkl/mkl_conv_grad_filter_ops.cc:638


AbortedError: Graph execution error:

Detected at node 'gradients/resnet_v1_50/block1/unit_1/bottleneck_v1/conv2/Conv2D_grad/Conv2DBackpropFilter' defined at (most recent call last):
    File "/home/asus/anaconda3/envs/DeepLabCut/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/asus/anaconda3/envs/DeepLabCut/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/home/asus/anaconda3/envs/DeepLabCut/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "/home/asus/anaconda3/envs/DeepLabCut/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance
      app.start()
    File "/home/asus/anaconda3/envs/DeepLabCut/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start
      self.io_loop.start()
    File "/home/asus/anaconda3/envs/DeepLabCut/lib/python3.10/site-packages/tor

We recommend you run this for ~1,000 iterations, just as a demo. This should take around 20 min. Note, that **when you hit "STOP" you will get a KeyInterrupt "error"! No worries! :)**

## Start evaluating:
This function evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images)
and stores the results as .csv file in a subdirectory under **evaluation-results**

In [6]:
%matplotlib notebook
deeplabcut.evaluate_network(path_config_file,plotting=True)

# Here you want to see a low pixel error! Of course, it can only be as good as the labeler, so be sure your labels are good!

Config:
{'all_joints': [[0], [1], [2], [3]],
 'all_joints_names': ['bodypart1', 'bodypart2', 'bodypart3', 'objectA'],
 'batch_size': 1,
 'crop_pad': 0,
 'dataset': 'training-datasets/iteration-0/UnaugmentedDataSet_MouseTestFeb22/MouseTest_Wynne95shuffle1.mat',
 'dataset_type': 'imgaug',
 'deterministic': False,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': '/home/asus/anaconda3/envs/DeepLabCut/lib/python3.10/site-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weight': 1.0,
 'locref_stdev': 7.2801,
 'log_dir': 'log',
 'mean_pixel': [123.68, 116.779, 103.939],
 'mirror': False,
 'net_type': 'resnet_50',
 'num_joints': 4,
 'optimizer': 'sgd',
 'pairwise_huber_loss': True,
 'pairwise_predict': False,
 'partaffinityfield_predict': False,
 'regularize': False,
 'scoremap_dir': 'test',
 'shuffle'

Running  DLC_resnet50_MouseTestFeb22shuffle1_200  with # of training iterations: 200
Running evaluation ...


19it [00:12,  1.49it/s]

Analysis is done and the results are stored (see evaluation-results) for snapshot:  snapshot-200
Results for 200  training iterations: 95 1 train error: 104.97 pixels. Test error: 649.48  pixels.
With pcutoff of 0.6  train error: nan pixels. Test error: nan pixels
Thereby, the errors are given by the average distances between the labels by DLC and the scorer.
Plotting...



/home/asus/anaconda3/envs/DeepLabCut/lib/python3.10/site-packages/deeplabcut/pose_estimation_tensorflow/core/evaluate.py:889: FutureWarning: Starting with pandas version 3.0 all arguments of to_hdf except for the argument 'path_or_buf' will be keyword-only.
  DataMachine.to_hdf(resultsfilename, "df_with_missing")


<IPython.core.display.Javascript object>

  0%|          | 0/19 [00:00<?, ?it/s]/home/asus/anaconda3/envs/DeepLabCut/lib/python3.10/site-packages/deeplabcut/utils/visualization.py:67: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  DataCombined[loopscorer][bp]["y"][imagenr]
/home/asus/anaconda3/envs/DeepLabCut/lib/python3.10/site-packages/deeplabcut/utils/visualization.py:68: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  + DataCombined[loopscorer][bp]["x"][imagenr]
/home/asus/anaconda3/envs/DeepLabCut/lib/python3.10/site-packages/deeplabcut/utils/visualization.py:71: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future versio

The network is evaluated and the results are stored in the subdirectory 'evaluation_results'.
Please check the results, then choose the best model (snapshot) for prediction. You can update the config.yaml file with the appropriate index for the 'snapshotindex'.
Use the function 'analyze_video' to make predictions on new videos.
Otherwise, consider adding more labeled-data and retraining the network (see DeepLabCut workflow Fig 2, Nath 2019)


**Check the images**:
You can go look in the newly created "evalutaion-results" folder at the images. At around 3500 iterations, the error is ~3 pixels (but this can vary on how your demo data was split for training)

## Start Analyzing videos:
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in hd5 file in the same directory where the video resides.

**On the demo data, this should take around ~ 3 min! (The demo frames are 640x480, which should run around 35 FPS on the google-provided GPU)**

In [7]:
videofile_path = ['/home/asus/Desktop/MouseTest-Wynne-2025-02-22/videos/SH301.MOV'] #Enter the list of videos to analyze.
deeplabcut.analyze_videos(path_config_file,videofile_path, videotype='.mov')

Config:
{'all_joints': [[0], [1], [2], [3]],
 'all_joints_names': ['bodypart1', 'bodypart2', 'bodypart3', 'objectA'],
 'batch_size': 1,
 'crop_pad': 0,
 'dataset': 'training-datasets/iteration-0/UnaugmentedDataSet_MouseTestFeb22/MouseTest_Wynne95shuffle1.mat',
 'dataset_type': 'imgaug',
 'deterministic': False,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': '/home/asus/anaconda3/envs/DeepLabCut/lib/python3.10/site-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weight': 1.0,
 'locref_stdev': 7.2801,
 'log_dir': 'log',
 'mean_pixel': [123.68, 116.779, 103.939],
 'mirror': False,
 'net_type': 'resnet_50',
 'num_joints': 4,
 'optimizer': 'sgd',
 'pairwise_huber_loss': True,
 'pairwise_predict': False,
 'partaffinityfield_predict': False,
 'regularize': False,
 'scoremap_dir': 'test',
 'shuffle'

Using snapshot-200 for model /home/asus/Desktop/MouseTest-Wynne-2025-02-22/dlc-models/iteration-0/MouseTestFeb22-trainset95shuffle1
No video(s) were found. Please check your paths and/or 'video_type'.


'DLC_resnet50_MouseTestFeb22shuffle1_200'

## Create labeled video:
This function is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides. This should run around 215 FPS on the demo video!

In [8]:
deeplabcut.create_labeled_video(path_config_file,videofile_path)

[]

In [17]:
deeplabcut.create_labeled_video(
    config=path_config_file,
    videos=videofile_path,
    videotype=".mp4",  # 视频类型
    shuffle=1,  # 使用哪个模型（默认为1）
    save_frames=False,  # 是否保存帧
    draw_skeleton=True,  # 是否绘制骨架
    destfolder=videofile_path,  # 指定输出目录
    pcutoff=0.6,  # 置信度阈值
    dotsize=8,  # 关键点大小
    colormap="rainbow",  # 颜色映射
)

TypeError: expected str, bytes or os.PathLike object, not list

## Plot the trajectories of the analyzed videos:
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [19]:
deeplabcut.plot_trajectories(path_config_file,videofile_path)

Loading  /home/asus/Downloads/cloned-DLC-repo/examples/openfield-Pranav-2018-10-30/videos/m3v1mp4.mp4 and data.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Plots created! Please check the directory "plot-poses" within the video directory


In [5]:
import deeplabcut
deeplabcut.gui()

AttributeError: module 'deeplabcut' has no attribute 'gui'